In [0]:
import tensorflow as tf
from tensorflow.python.saved_model import tag_constants
import numpy as np
import time, random
import matplotlib.pyplot as plt
from google.colab import files
import warnings

In [0]:
warnings.simplefilter("ignore", DeprecationWarning)
warnings.simplefilter("ignore", FutureWarning)

In [0]:
#DATASET UPLOADS
upload = files.upload()

In [0]:
#NECESSARY TO ACCESS UPLOADED FILES, IDK WHY
for k,v in uploaded.keys():
  open(k,'wb').write(v)

In [0]:
training_images = np.load('emnist_digits_training_images.npy')
training_labels = np.load('emnist_digits_training_labels.npy')
test_images = np.load('emnist_digits_test_images.npy')
test_labels = np.load('emnist_digits_test_labels.npy')

In [0]:
def get_one_hot(labels,w,ls):
  return np.eye(10)[labels.astype(int)] #.astype(int)

def loss2(logits,onehotlabs,w,ls):
  return tf.losses.softmax_cross_entropy(onehotlabs,logits,weights=w,label_smoothing=ls,reduction=None)

def aux_loss(logits,labels):
  return tf.reduce_mean(loss2(logits,get_one_hot(labels),0.4,0.1),name='aux_cross_entropy')

def loss(aux_logits,logits,labels):
	labels = tf.reshape(tf.cast(labels,tf.int64),[-1])
	#cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels,logits=logits,name='cross_entropy_per_example')
	cross_entropy_mean = tf.reduce_mean(loss2(logits,get_one_hot(labels),1.0,0.1),name='final_cross_entropy')
  
  total_ce_mean = tf.add(aux_loss(aux_logits,labels),cross_entropy_mean,name='aux_plus_final')
	
  total_loss = tf.add(tf.reduce_sum(tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)),total_ce_mean,name='total_loss')
	
  return total_loss

def accuracy(logits,true_labels):
	pred_labels = tf.argmax(logits,1)
	true_labels = tf.cast(true_labels,tf.int64)
	#print pred_labels.get_shape().as_list(),true_labels
	correct_pred = tf.cast(tf.equal(pred_labels, true_labels), tf.float32)
	accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))
	return accuracy

In [0]:
x = tf.placeholder(tf.float32,shape=[None,28,28],name='x')
y = tf.placeholder(tf.int64,shape=[None],name='y')
keep_prob = tf.placeholder(tf.float32,shape=(),name='keep_prob')
#lr = tf.placeholder(tf.float32,shape=())
mode = tf.placeholder(tf.bool,shape=(),name='mode')

aug_img = tf.placeholder(tf.float32,shape=[28,28],name='aug_img')

In [0]:
img_rotate = tf.contrib.image.rotate(aug_img,-0.5+np.random.random(),interpolation='BILINEAR')
img_affine = tf.contrib.image.transform(aug_img,[np.random.random() for i in xrange(8)])

In [0]:
#INCEPTION MODULE A
def inceptionA(name,base_inp):
  with tf.variable_scope(name):
    br1 = tf.layers.conv2d(base_inp,filters=64,kernel_size=1,strides=1,padding='same',activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005,dtype=tf.float32)),name='b1_1x1')
    br1 = tf.layers.conv2d(br1,filters=96,kernel_size=3,strides=1,padding='same',activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005,dtype=tf.float32)),name='b1_3x3_1')
    br1 = tf.layers.conv2d(br1,filters=96,kernel_size=3,strides=1,padding='same',activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005,dtype=tf.float32)),name='b1_3x3_2')
    
    br2 = tf.layers.conv2d(base_inp,filters=48,kernel_size=1,strides=1,padding='same',activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005,dtype=tf.float32)),name='b2_1x1')
    br2 = tf.layers.conv2d(br2,filters=64,kernel_size=3,strides=1,padding='same',activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005,dtype=tf.float32)),name='b2_3x3')
    
    br3 = tf.layers.average_pooling2d(base_inp,pool_size=3,strides=1,padding='same',name='incA_b3_avg_pool')
    br3 = tf.layers.conv2d(br3,filters=64,kernel_size=3,strides=1,padding='same',activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005,dtype=tf.float32)),name='b3_1x1')
    
    br4 = tf.layers.conv2d(base_inp,filters=64,kernel_size=1,strides=1,padding='same',activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005,dtype=tf.float32)),name='b4_1x1')
    
    br_concat = tf.concat([br1,br2,br3,br4],3, name+'concat')
    
    return branch_concat

In [0]:
#INCEPTION MODULE B
def inceptionB(name,base_inp):
  with tf.variable_scope(name):
    br1 = tf.layers.conv2d(base_inp,filters=160,kernel_size=1,strides=1,padding='same',activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005,dtype=tf.float32)),name='b1_1x1')
    br1 = tf.layers.conv2d(br1,filters=160,kernel_size=[3,1],strides=1,padding='same',activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005,dtype=tf.float32)),name='b1_3x1_1')
    br1 = tf.layers.conv2d(br1,filters=160,kernel_size=[1,3],strides=1,padding='same',activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005,dtype=tf.float32)),name='b1_1x3_1')
    br1 = tf.layers.conv2d(br1,filters=160,kernel_size=[3,1],strides=1,padding='same',activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005,dtype=tf.float32)),name='b1_3x1_2')
    br1 = tf.layers.conv2d(br1,filters=192,kernel_size=[1,3],strides=1,padding='same',activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005,dtype=tf.float32)),name='b1_1x3_2')
    
    br2 = tf.layers.conv2d(base_inp,filters=160,kernel_size=1,strides=1,padding='same',activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005,dtype=tf.float32)),name='b2_1x1')
    br2 = tf.layers.conv2d(br2,filters=160,kernel_size=[3,1],strides=1,padding='same',activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005,dtype=tf.float32)),name='b2_3x1')
    br2 = tf.layers.conv2d(br2,filters=192,kernel_size=[1,3],strides=1,padding='same',activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005,dtype=tf.float32)),name='b2_1x3')
    
    br3 = tf.layers.average_pooling2d(base_inp,pool_size=3,strides=1,padding='same',name='incB_b3_avg_pool')
    br3 = tf.layers.conv2d(br3,filters=192,kernel_size=1,strides=1,padding='same',activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005,dtype=tf.float32)),name='b3_1x1')
    
    br4 = tf.layers.conv2d(base_inp,filters=192,kernel_size=1,strides=1,padding='same',activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005,dtype=tf.float32)),name='b4_1x1')
    
    br_concat = tf.concat([br1,br2,br3,br4],3,name=name+'concat')
    
    return br_concat

In [0]:
#INCEPTION MODULE C
def inceptionC(name,base_inp):
  with tf.variable_scope(name):
    br1 = tf.layers.conv2d(base_inp,filters=448,kernel_size=1,strides=1,padding='same',activation=tf.nn.relu,krenel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer()tf.constant(0.0005,dtype=tf.float32),name='b1_1x1')
    br1 = tf.layers.conv2d(br1,filters=384,kernel_size=3,strides=1,padding='same',activation=tf.nn.relu,krenel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer()tf.constant(0.0005,dtype=tf.float32),name='b1_3x3')
    br1_1 = tf.layers.conv2d(br1,filters=384,kernel_size=[1,3],strides=1,padding='same',activation=tf.nn.relu,krenel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer()tf.constant(0.0005,dtype=tf.float32),name='b1_1x3')
    br1_2 = tf.layers.conv2d(br1,filters=384,kernel_size=[3,1],strides=1,padding='same',activation=tf.nn.relu,krenel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer()tf.constant(0.0005,dtype=tf.float32),name='b1_3x1')
    
    br2 = tf.layers.conv2d(base_inp,filters=384,kernel_size=1,strides=1,padding='same',activation=tf.nn.relu,krenel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer()tf.constant(0.0005,dtype=tf.float32),name='b2_1x1')
    br2_1 = tf.layers.conv2d(br2,filters=384,kernel_size=[1,3],strides=1,padding='same',activation=tf.nn.relu,krenel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer()tf.constant(0.0005,dtype=tf.float32),name='b2_1x3')
    br2_2 = tf.layers.conv2d(br2,filters=384,kernel_size=[3,1],strides=1,padding='same',activation=tf.nn.relu,krenel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer()tf.constant(0.0005,dtype=tf.float32),name='b2_3x1')
    
    br3 = tf.layers.average_pooling2d(base_inp,pool_size=3,strides=1,padding='same',name='incC_b3_avg_pool')
    br3 = tf.layers.conv2d(br3,filters=192,kernel_size=1,strides=1,padding='same',activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005,dtype=tf.float32)),name='b3_1x1')
    
    br4 = tf.layers.conv2d(base_inp,filters=320,kernel_size=1,strides=1,padding='same',activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005,dtype=tf.float32)),name='b4_1x1')
    
    br_concat = tf.concat([br1_1,br1_2,br2_1,br2_2,br3,br4],3,name=name+'concat')
    
    return br_concat

In [0]:
#INCEPTION-V2
with tf.device('/gpu:0'):
  #STEM
  #28X28
  conv1 = tf.layers.conv2d(tf.reshape(x,shape=[-1,28,28,1]),filters=32,kernel_size=3,strides=1,padding='same',activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005,dtype=tf.float32)),name='conv1')
  conv2 = tf.layers.conv2d(conv1,filters=32,kernel_size=3,strides=1,padding='same',activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005,dtype=tf.float32)),name='conv2')
  conv3 = tf.layers.conv2d(conv2,filters=64,kernel_size=3,strides=1,padding='same',activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005,dtype=tf.float32)),name='conv3')
  pool1 = tf.layers.max_pooling2d(conv3,pool_size=3,strides=1,padding='same',name='pool1')
  #28X28X64
  conv4 = tf.layers.conv2d(pool1,filters=80,kernel_size=3,strides=1,padding='same',activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005,dtype=tf.float32)),name='conv4')
  conv5 = tf.layers.conv2d(conv4,filters=192,kernel_size=3,strides=1,padding='same',activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005,dtype=tf.float32)),name='conv5')
  conv6 = tf.layers.conv2d(conv5,filters=256,kernel_size=3,strides=1,padding='same',activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005,dtype=tf.float32)),name='conv6')
  #28X28X256
  #Inception Module A
  incA1 = inceptionA('incA1',conv6)
  incA2 = inceptionA('incA2',incA1)
  incA3 = inceptionA('incA3',incA2)
  #28X28X288
  #GRID SIZE REDUCTION
  post_incA_b1 = tf.layers.conv2d(incA3,filters=384,kernel_size=3,strides=2,padding='valid',activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005,dtype=tf.float32)),name='post_incA_b1')
  
  post_incA_b2 = tf.layers.conv2d(incA3,filters=64,kernel_size=1,strides=1,padding='same',activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005,dtype=tf.float32)),name='post_incA_b2_1x1')
  post_incA_b2 = tf.layers.conv2d(post_incA_b2,filters=96,kernel_size=3,strides=1,padding='same',activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005,dtype=tf.float32)),name='post_incA_b2_3x3_1')
  post_incA_b2 = tf.layers.conv2d(post_incA_b2,filters=96,kernel_size=3,strides=2,padding='valid',activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005,dtype=tf.float32)),name='post_incA_b2_3x3_2')
  
  post_incA_b3 = tf.layers.max_pooling2d(incA3,pool_size=3,strides=2,padding='valid',name='post_incA_maxpool')
  
  incB_base = tf.concat([post_incA_b1,post_incA_b2,post_incA_b3],axis=3,name='incB_base')
  #14X14X768
  #Inception Module B
  incB1 = inceptionB('incB1',incB_base)
  incB2 = inceptionB('incB2',incB1)
  incB3 = inceptionB('incB3',incB2)
  #incB4 = inceptionB('incB4',incB3)
  #incB5 = inceptionB('incB5',incB4)
  #14X14X768
  
  #AUXILIARY SOFTMAX
  #10X1
  aux_head = tf.layers.average_pooling2d(incB3,pool_size=3,strides=2,padding='valid',name='aux_head_avg_pool')
  #6X6X768
  aux_head = tf.layers.conv2d(aux_head,filters=128,kernel_size=1,strides=1,padding='same',activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005,dtype=tf.float32)),name='aux_head_1x1')
  #6X6X128
  aux_head = tf.layers.conv2d(aux_head,filters=384,kernel_size=3,strides=2,padding='valid',activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005,dtype=tf.float32)),name='aux_head_3x3')
  #2X2X384
  aux_head = tf.layers.dense(aux_head,units=768,activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initiaizer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005,dtype=tf.float32)),name='aux_head_fc')
  #768X1
  aux_logits = tf.layers.dense(aux_head,units=10,activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initiaizer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005,dtype=tf.float32)),name='aux_head_logits')
  #10X1
  
  #GRID SIZE REDUCTION
  post_incB_b1 = tf.layers.conv2d(incB3,filters=192,kernel_size=1,strides=1,padding='same',activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005,dtype=tf.float32)),name='post_incB_b1_1x1')
  post_incB_b1 = tf.layers.conv2d(post_incB_b1,filters=320,kernel_size=3,strides=2,padding='valid',activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005,dtype=tf.float32)),name='post_incB_b1_3x3')
  
  post_incB_b2 = tf.layers.conv2d(incB3,filters=192,kernel_size=1,strides=1,padding='same',activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005,dtype=tf.float32)),name='post_incB_b2_1x1')
  post_incB_b2 = tf.layers.conv2d(post_incB_b2,filters=192,kernel_size=[1,3],strides=1,padding='same',activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005,dtype=tf.float32)),name='post_incB_b2_1x3')
  post_incB_b2 = tf.layers.conv2d(post_incB_b2,filters=192,kernel_size=[3,1],strides=1,padding='same',activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005,dtype=tf.float32)),name='post_incB_b2_3x1')
  post_incB_b2 = tf.layers.conv2d(post_incB_b2,filters=192,kernel_size=3,strides=2,padding='valid',activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005,dtype=tf.float32)),name='post_incB_b2_3x3')
  
  post_incB_b3 = tf.layers.max_pooling2d(incB3,pool_size=3,strides=2,padding='valid',name='post_incB_max_pool')
  
  incC_base = tf.concat([post_incB_b1,post_incB_b2,post_incB_b3],3,name='incC_base')
  #7X7X1280
  #Inception Module C
  incC1 = inceptionC('incC1',incC_base)
  incC2 = inceptionC('incC2',incC1)
  #incC3 = inceptionC('incC3',incC2)
  #7X7X2048
  pool2 = tf.layers.average_pooling2d(incC2,pool_size=7,strides=1,padding='valid',name='pool2_avg_pool')
  #1X1X2048
  pool2_flatten = tf.layers.flatten(pool2)
  dropout1 = tf.layers.dropout(pool2,rate=keep_prob,training=mode) #introduce new placeholder
  logits = tf.layers.dense(dropout1,units=10)
  

In [0]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [0]:
epochs = 25
ne = 0
tracc=[]
valacc=[]
trerr=[]
valerr=[]
batch_size = 100
numiter = 2000
index=np.arange(200000)
index_t = np.arange(40000)

with tf.device('/gpu:0'):
  while(ne<epochs):
    print 'Epoch:: ',ne+1,'-->'
    stime = time.time()
    if ne != 0:
		  np.random.shuffle(index)
		  images = images[index]
		  labels = labels[index]
    for niter in xrange(numiter):
      #if (niter+1)%100==0:
        #print 'Iter::',niter+1,'---------'
      offset = niter*batch_size
      x_iter, y_iter = np.array(images[offset:offset+batch_size,:,:]), np.array(labels[offset:offset+batch_size])
      
      #Training Dict
      feed_trdict={x:x_iter,y:y_iter,keep_prob:0.8,mode:True}
		  
      #Train
      sess.run(opt_op,feed_dict=feed_trdict)
    
    
    #Calculate accuracy of Training set
    print 'Calculating Loss and Accuracy...'
    tr_loss = sess.run(cost,feed_dict=feed_trdict)
    tr_acc = sess.run(top1_acc,feed_dict = feed_trdict)
    
    #Calculate accuracy of Validation set
    np.random.shuffle(index_t)
    val_loss = 0
    val_acc = 0
    for j in xrange(400):
      val_loss += sess.run(cost,feed_dict = {x:test_imgs[index_t[j*100:(j+1)*100]],y:test_labels[index_t[j*100:(j+1)*100]],keep_prob:1.0,mode:False})
      val_acc += sess.run(top1_acc,feed_dict = {x:test_imgs[index_t[j*100:(j+1)*100]],y:test_labels[index_t[j*100:(j+1)*100]],keep_prob:1.0,mode:False})
    val_loss/=400
    val_acc/=400
    print 'Appending values'
    valacc.append(val_acc)
    tracc.append(tr_acc)
    trerr.append(tr_loss)
    valerr.append(val_loss)
    print 'Epoch..',ne+1,'...'
    print 'Training cost::',tr_loss,' Validation Cost:: ',val_loss
    print 'Training accuracy::',tr_acc*100,'%',' Validation accuracy::',val_acc*100,'%'
    print 'Time reqd.::',(time.time()-stime)/60,'mins...'
    ne = ne + 1

In [0]:
plt.plot(tracc)
plt.plot(valacc)
plt.show()

In [0]:
plt.plot(trerr)
plt.plot(valerr)
plt.show()

In [0]:
#INFERENCE
test_acc=[]
tindex = np.arange(40000)

for nepoch in xrange(100):
  stime = time.time()
	if nepoch !=0:
		np.random.shuffle(tindex)
		test_x = test_x[index]
		test_y = test_y[index]
	test_loss = sess.run(cost,feed_dict = {feed_images:test_x,feed_labels:test_y,keep_prob:1.0,mode=False})
	test_acc = sess.run(acc,feed_dict = {feed_images:test_x,feed_labels:test_y,keep_prob:1.0,mode=False})
	print 'Epoch..',ne+1,'...'
	print 'Time reqd.::',(time.time()-stime)/60,'mins...'
  test_acc.append(test_acc)

In [0]:
plt.plot(test_acc)

In [0]:
#SAVE MODEL
graph = tf.get_default_graph()

with graph.as_default():
  input_dict = {"x":x,"y":y,"keep_prob":keep_prob,"aug_img":aug_img,"mode":mode}
  output_dict = {"logits":logits,"aux_logits":aux_logits}
  tf.saved_model.simple_save(sess,'/mnist_saved_model/',input_dict,output_dict)

In [0]:
sess.close()